In [45]:
import re
import pandas as pd

# Define file paths
excel_file_path = 'output_hamrobazaar/initial_raw_data.xlsx'
output_file_path = 'output_hamrobazaar/3.cleaned_data.xlsx'

# Read the Excel file
dfs = pd.read_excel(excel_file_path, sheet_name=None)

# Columns to select
selected_columns = ['PRICE', 'KMRANGE', 'BODYTYPE', 'BRANDNAME','MAKEYEAR','ENGINE','COLOR','TRANSMISSION','FUEL','CREATEDON','ID']  # Adjust as necessary

# Function to clean and convert the 'PRICE' column
def clean_price(price):
    return int(str(price).replace('NPR', '').replace(',', '')) if pd.notna(price) else price

# Function to clean and convert 'KMRANGE' column
def clean_kmrange(kmrange):
    if pd.isna(kmrange) or (isinstance(kmrange, str) and kmrange.strip() in ['', 'Null']):
        return None

    # Remove any non-numeric characters
    if isinstance(kmrange, str):
        # Use regular expression to extract numeric characters only
        kmrange = re.sub(r'[^\d]', '', kmrange)
    
    # Convert to integer if possible
    try:
        kmrange = int(kmrange)
    except (ValueError, TypeError):
        return None
    
    # If the value is less than 100, multiply it by 1000
    if kmrange < 199:
        kmrange *= 1000
    if kmrange>800000:
        return None
    if kmrange ==0:
        return None
    
    return kmrange

# Define regular expression patterns for transmission and body type
automatic_pattern = re.compile(r'(Auto|Automatic|Power\s*Steering|PowerSteering)', flags=re.IGNORECASE)
manual_pattern = re.compile(r'Manual', flags=re.IGNORECASE)
drive_pattern = re.compile(r'(2WD|4WD|4 WD|2 WD)', flags=re.IGNORECASE)
SUV_pattern = re.compile(r'SUV', flags=re.IGNORECASE)
HatchBack_pattern = re.compile(r'Hatchback', flags=re.IGNORECASE)

# Create a writer for the new Excel file
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    
    # Iterate through sheets and process
    for sheet_name, df in dfs.items():
        if sheet_name != 'All Data':
            df[selected_columns].to_excel(writer, sheet_name=sheet_name, index=False)
    
    # Process 'All Data' sheet
    all_data_sheet = next(iter(dfs.values()), pd.DataFrame())

    if not all_data_sheet.empty:
        all_data_sheet_copy = all_data_sheet[selected_columns].copy()

        # Clean 'PRICE' column
        if 'PRICE' in all_data_sheet_copy.columns:
            all_data_sheet_copy['PRICE'] = all_data_sheet_copy['PRICE'].apply(clean_price)

        # Clean 'KMRANGE' column
        if 'KMRANGE' in all_data_sheet_copy.columns:
            all_data_sheet_copy['MILEAGE'] = all_data_sheet_copy['KMRANGE'].apply(clean_kmrange)

        # Clean 'TRANSMISSION' column
        if 'TRANSMISSION' in all_data_sheet_copy.columns:
            all_data_sheet_copy['GEARBOX'] = all_data_sheet_copy['TRANSMISSION'].apply(
                lambda x: 'Automatic' if pd.notna(x) and automatic_pattern.search(str(x)) else 
                          'Manual' if pd.notna(x) and manual_pattern.search(str(x)) else '-'
            )
            all_data_sheet_copy['DRIVE'] = (all_data_sheet_copy['TRANSMISSION']
                                            .apply(lambda x: drive_pattern.search(str(x)) and drive_pattern.search(str(x)).group(0)))
            all_data_sheet_copy['DRIVE'] = all_data_sheet_copy['DRIVE'].str.replace(" ", "")

        # Clean 'BODYTYPE' column
        if 'BODYTYPE' in all_data_sheet_copy.columns:
            all_data_sheet_copy['BODY'] = all_data_sheet_copy['BODYTYPE'].apply(
                lambda x: 'SUV' if pd.notna(x) and SUV_pattern.search(str(x)) else 
                          'Hatchback' if pd.notna(x) and HatchBack_pattern.search(str(x)) else x
            )
       

        # # Filter rows to keep only 'Petrol' and 'Diesel' fuel types
        # if 'FUEL' in all_data_sheet_copy.columns:
        #     # Remove rows where FUEL is NaN
        #     all_data_sheet_copy = all_data_sheet_copy.dropna(subset=['FUEL'])
            
        #     # Keep rows where FUEL is either 'Petrol' or 'Diesel'
        #     all_data_sheet_copy = all_data_sheet_copy[all_data_sheet_copy['FUEL'].isin(['Petrol', 'Diesel'])] else x

        # Drop unnecessary columns
        all_data_sheet_copy = all_data_sheet_copy.drop(columns=['BODYTYPE', 'TRANSMISSION', 'KMRANGE'], errors='ignore')

        # Write modified 'All Data' to new sheet
        all_data_sheet_copy.to_excel(writer, sheet_name='Data', index=False)

print("Done!")


Done!


In [51]:
import pandas as pd

# Define file paths
input_file_path = 'output_hamrobazaar/3.cleaned_data.xlsx'
output_file_path = 'output_hamrobazaar/4.final_raw_data.xlsx'

# Read the Excel file
df = pd.read_excel(input_file_path,sheet_name='Data')

# Conditions to filter the rows
fuel_condition = df['FUEL'].isin(['Petrol', 'Diesel'])
drive_condition = df['DRIVE'].isin(['4WD', '2WD'])
gearbox_condition = df['GEARBOX'].isin(['Automatic', 'Manual'])
df['CREATEDON'] = pd.to_datetime(df['CREATEDON'], errors='coerce')



df['MAKEYEAR'] = pd.to_numeric(df['MAKEYEAR'], errors='coerce')
df['ENGINE'] = pd.to_numeric(df['ENGINE'], errors='coerce')  # Convert to numeric, NaN if fails

# Apply filters to keep only rows that meet all the conditions
filtered_df = df[fuel_condition & drive_condition & gearbox_condition]
filtered_df = filtered_df.dropna()

filtered_df['AGE'] = filtered_df['CREATEDON'].dt.year - filtered_df['MAKEYEAR'].astype(int)

bins = [0, 1000, 1500, 2000, 2500, 2900, float('inf')]
labels = ['0-1000(cc)', '1001-1500(cc)', '1501-2000(cc)', '2001-2500(cc)', '2501-2900(cc)', '2901-above(cc)']
filtered_df['ENGINERANGE'] =  pd.cut(filtered_df['ENGINE'], bins=bins, labels=labels, right=True)

# Save the filtered data to a new Excel file
filtered_df.to_excel(output_file_path, index=False)

print("Filtered data saved to", output_file_path)

Filtered data saved to output_hamrobazaar/4.final_raw_data.xlsx


In [70]:
# Merging Data for final Data sets 
import re
import pandas as pd
import os

# Define file paths
recent_file_path = 'output_hamrobazaar/4.final_raw_data.xlsx'
earlier_file_path = 'output_from_thesis/Raw-Data-Final.csv'
output_dir='final_output'
output_file_path = os.path.join(output_dir, 'final_raw_dataset.xlsx')

# Read the Excel file
df1 = pd.read_excel(recent_file_path)
df2 = pd.read_csv(earlier_file_path)
selected_columns = ['PRICE', 'MILEAGE', 'BRANDNAME','AGE', 'BODY', 'FUEL', 'GEARBOX', 'DRIVE', 'COLOR', 'ENGINERANGE']
df_selected = df1[selected_columns]
selected_columns2 = ['PRICE_NEW', 'KM_INT', 'BRANDNAME','AGE', 'BODY', 'FUEL', 'GEARBOX', 'DRIVE', 'COLOR', 'ENGINERANGE']
df2_selected=df2[selected_columns2]

df2 = df2_selected.rename(columns={
    'PRICE_NEW': 'PRICE',
    'KM_INT': 'MILEAGE',
})
combined_df = pd.concat([df_selected, df2], ignore_index=True)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
    combined_df.to_excel(writer, sheet_name='MergedData', index=False)

    print("Final Raw data saved to", output_file_path)

Final Raw data saved to final_output/final_raw_dataset.xlsx


https://www.youtube.com/watch?v=Wqmtf9SA_kk&ab_channel=NeuralNine
https://www.youtube.com/watch?v=xu0prYu5e-c&ab_channel=AliNasserEddine